In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import os

In [ ]:

# 1. Simulate Keylogger Behavior Data
def load_dataset(path="keylogger_data.csv"):
    if not os.path.exists(path):
        print("Dataset not found. Creating synthetic keylogger detection dataset...")

        data = {
            "keystroke_rate": np.random.uniform(0, 20, 1000),          # keys/sec
            "screenshot_freq": np.random.randint(0, 2, 1000),          # 0/1
            "file_access_rate": np.random.uniform(0, 30, 1000),        # files/min
            "network_bytes_sent": np.random.randint(0, 50000, 1000),
            "is_hidden_process": np.random.randint(0, 2, 1000),
            "process_duration": np.random.uniform(1, 300, 1000),       # seconds
            "is_keylogger": np.random.randint(0, 2, 1000)
        }

        df = pd.DataFrame(data)
        df.to_csv(path, index=False)
    else:
        df = pd.read_csv(path)

    print(f"Loaded dataset with shape: {df.shape}")
    return df
